In [ ]:
# from CoRe_Dataloader_ECSG import *
from CoRe_Dataloader_From_Files_With_OvGN import *
import time
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
import os

In [ ]:
# btime = time.time()
# len_dataset = len(dataset)
# print("finished with Initialization")
# print("pspace: ", len(dataset.pspace))
# spectrograms = []
# params = []
# for i, (spectrogram, param) in enumerate(dataset):  # type: ignore
#     print(i, len_dataset, spectrogram.shape, (time.time() - btime) / 60)
#     spectrograms.append(spectrogram.cpu())
#     params.append(param.cpu()) 
# comb = list(zip(spectrograms, params))
# len(comb)
# def numshift_pad_width(Z, l=FIN_WIDTH, shift=0):
#     cwidth = Z.shape[1]
#     leftpad = int((l - cwidth) / 2)
#     rightpad = int((l - cwidth) / 2)
#     fudgepad = rightpad + int(l - (leftpad + rightpad + cwidth))
#     # print(leftpad, rightpad, fudgepad)
#     assert (shift <= leftpad) and (
#         shift <= rightpad
#     ), "Shift must be less than or equal to the padding on either side"
#     padb = np.zeros((l, leftpad + shift))
#     pada = np.zeros((l, fudgepad - shift))
#     return np.concatenate((padb, Z, pada), axis=1)


# def pad_width(Z,  percent_shift=0,l=FIN_WIDTH):
#     maxshift = int((l - Z.shape[1]) / 2)
#     # print(maxshift)
#     shift = int(percent_shift * maxshift * (1/100)) if percent_shift != 0 else 0
#     return numshift_pad_width(Z, shift=shift)
# ! del /A /Q padded_spectrograms
# ctr = 0
# for i, (sgram, params) in enumerate(comb):
#     for shift in range(-100,101,10):
#         padded_spectrogram = torch.tensor(pad_width(sgram, shift))
#         name = f"./padded_spectrograms/{i}_{shift}_{params[0].item()}_{params[1].item()}_{params[2].item()}.pt"
#         torch.save((padded_spectrogram, torch.concat((params, torch.tensor([shift])))), name)
#         ctr += 1
#         print(i,shift,ctr,params,torch.concat((params, torch.tensor([shift]))), end="\r\r")
#     print("\nFinished with ", i, ctr,params,"                "*4)
# len(os.listdir("./padded_spectrograms/"))


In [ ]:
tsgram, tparams = torch.load("./padded_spectrograms/" + os.listdir("./padded_spectrograms/")[-1])

In [ ]:
plt.pcolormesh(tsgram)
plt.title("Spectrogram")
plt.colorbar()
plt.show()

std = calculate_std(tsgram,0)
noise = torch.normal(0, std, tsgram.shape)
plt.pcolormesh(noise)
plt.title("Noise")
print(std,noise.shape,torch.max(noise))
plt.colorbar()
plt.show()

plt.pcolormesh(noise+tsgram)
plt.title("noise + signal")
plt.colorbar()
plt.show()

In [ ]:
nsgram = tsgram /torch.max(tsgram)
plt.pcolormesh(nsgram)
plt.colorbar()
plt.show()
std = calculate_std(nsgram,.5)
noise = torch.normal(0, std, nsgram.shape)
noise.shape
plt.pcolormesh(noise)
plt.colorbar()
plt.show()
plt.pcolormesh(noise+nsgram)
plt.colorbar()

In [1]:
from CoRe_Dataloader_From_Files_With_OvGN import *
train_dl, valid_dl, test_dl = get_new_train_validation_test_dataloaders()


[['BAM:0051', 'R01', 'rh_22', 'Rh_l2_m2_r01000.txt', 0], ['THC:0006', 'R02', 'rh_22', 'Rh_l2_m2_r00400.txt', 0], ['BAM:0004', 'R02', 'rh_22', 'Rh_l2_m2_r01000.txt', 0], ['BAM:0100', 'R30', 'rh_22', 'Rh_l2_m2_r00850.txt', 0], ['BAM:0100', 'R10', 'rh_22', 'Rh_l2_m2_r00900.txt', 0], ['THC:0054', 'R02', 'rh_22', 'Rh_l2_m2_r00400.txt', 0], ['THC:0063', 'R03', 'rh_22', 'Rh_l2_m2_r00400.txt', 0], ['THC:0041', 'R02', 'rh_22', 'Rh_l2_m2_r00400.txt', 0], ['THC:0069', 'R02', 'rh_22', 'Rh_l2_m2_r00400.txt', 0], ['THC:0088', 'R03', 'rh_22', 'Rh_l2_m2_r00400.txt', 0], ['BAM:0039', 'R01', 'rh_22', 'Rh_l2_m2_r01000.txt', 0], ['THC:0046', 'R02', 'rh_22', 'Rh_l2_m2_r00400.txt', 0], ['BAM:0090', 'R01', 'rh_22', 'Rh_l2_m2_r01000.txt', 0], ['BAM:0100', 'R12', 'rh_22', 'Rh_l2_m2_r00900.txt', 0], ['THC:0102', 'R02', 'rh_22', 'Rh_l2_m2_r00400.txt', 0], ['BAM:0100', 'R07', 'rh_22', 'Rh_l2_m2_r00900.txt', 0], ['BAM:0037', 'R01', 'rh_22', 'Rh_l2_m2_r01000.txt', 0], ['BAM:0019', 'R01', 'rh_22', 'Rh_l2_m2_r01000.t

In [2]:
# import vit
# import vit_pytorch
from vit_pytorch import vit_for_small_dataset as vit_sd
from vit_pytorch import vit as simple_vit
from vit_pytorch.deepvit import DeepViT


def init_model():
    # return simple_vit.ViT(image_size=400,
    #                patch_size=20,
    #                num_classes=19,
    #                dim=int(1024/2),
    #                depth=2,
    #                heads=8,
    #                mlp_dim=int(2048/2),
    #                channels=1).to("cuda:0")
    # return vit_sd.ViT(image_size=400,
    #                patch_size=20,
    #                num_classes=19,
    #                dim=1024,
    #                depth=4,
    #                heads=16,
    #                mlp_dim=int(2048/2),
    #                dropout = 0.1,
    #                emb_dropout = 0,
    #                channels=1).to("cuda:0")
    return DeepViT(
        image_size=400,
        patch_size=20,
        num_classes=2,
        dim=1024,
        depth=4,
        heads=16,
        mlp_dim=int(2048 / 2),
        dropout=0.1,
        emb_dropout=0.1,
        channels=1,
    ).to("cuda:0")
tmodel = init_model()

In [3]:
for sgram, params in train_dl:
    print(torch.max(sgram),torch.min(sgram))
    sgram = torch.unsqueeze(sgram,1).to("cuda:0")
    print(tmodel(sgram.to("cuda:0").float()))
    input()

tensor(0.0154, device='cuda:0', dtype=torch.float64) tensor(-0.0004, device='cuda:0', dtype=torch.float64)
tensor([[ 0.1724,  0.6623],
        [ 0.0571,  1.1614],
        [ 0.3195,  0.8323],
        [ 0.0222,  0.7801],
        [ 0.2244,  0.8152],
        [ 0.0550,  0.8372],
        [-0.0333,  0.6081],
        [ 0.5950,  0.8708],
        [ 0.0774,  0.7624],
        [-0.1981,  0.5618],
        [ 0.3028,  0.7321],
        [ 0.2198,  0.6764],
        [ 0.1009,  0.9589],
        [ 0.5281,  0.9765],
        [ 0.0974,  0.4002],
        [ 0.1080,  0.4125]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor(0.0198, device='cuda:0', dtype=torch.float64) tensor(-0.0055, device='cuda:0', dtype=torch.float64)
tensor([[ 0.1483,  0.7134],
        [-0.2400,  0.4320],
        [ 0.0173,  0.2779],
        [ 0.1371,  0.8773],
        [ 0.0198,  0.4512],
        [ 0.0825,  0.5102],
        [-0.1570,  0.2276],
        [-0.1422,  0.6715],
        [ 0.5279,  0.3610],
        [-0.2811,  0.6494],
        [ 0.34

KeyboardInterrupt: Interrupted by user

In [4]:
print(params)

tensor([[ 1.4000e+01,  1.3500e+00,  1.3500e+00,  0.0000e+00,  3.2500e-04],
        [ 5.0000e+00,  1.6000e+00,  1.6000e+00,  0.0000e+00,  4.2500e-04],
        [ 1.6000e+01,  1.3500e+00,  1.3500e+00, -4.0000e+01,  2.0000e-04],
        [ 1.6000e+01,  1.5002e+00,  1.5002e+00,  5.0000e+01,  5.2500e-04],
        [ 2.0000e+00,  1.3755e+00,  1.3755e+00,  1.0000e+01,  4.5000e-04],
        [ 1.4000e+01,  1.3500e+00,  1.3500e+00,  0.0000e+00,  1.2500e-04],
        [ 6.0000e+00,  1.4000e+00,  1.4000e+00,  0.0000e+00,  2.7500e-04],
        [ 1.6000e+01,  1.3500e+00,  1.3500e+00,  7.0000e+01,  4.2500e-04],
        [ 2.0000e+00,  1.3500e+00,  1.3500e+00, -4.0000e+01,  3.7500e-04],
        [ 1.6000e+01,  1.3553e+00,  1.1072e+00,  4.0000e+01,  0.0000e+00],
        [ 1.6000e+01,  1.3500e+00,  1.3500e+00, -6.0000e+01,  5.0000e-05],
        [ 1.6000e+01,  1.3506e+00,  1.3506e+00,  4.0000e+01,  3.7500e-04],
        [ 1.6000e+01,  1.3500e+00,  1.3500e+00,  5.0000e+01,  3.0000e-04],
        [ 1.6000e+01,  1.

In [5]:
torch.tensor([0.001 * i / 200 for i in range(0, 201, 5)])

tensor([0.0000e+00, 2.5000e-05, 5.0000e-05, 7.5000e-05, 1.0000e-04, 1.2500e-04,
        1.5000e-04, 1.7500e-04, 2.0000e-04, 2.2500e-04, 2.5000e-04, 2.7500e-04,
        3.0000e-04, 3.2500e-04, 3.5000e-04, 3.7500e-04, 4.0000e-04, 4.2500e-04,
        4.5000e-04, 4.7500e-04, 5.0000e-04, 5.2500e-04, 5.5000e-04, 5.7500e-04,
        6.0000e-04, 6.2500e-04, 6.5000e-04, 6.7500e-04, 7.0000e-04, 7.2500e-04,
        7.5000e-04, 7.7500e-04, 8.0000e-04, 8.2500e-04, 8.5000e-04, 8.7500e-04,
        9.0000e-04, 9.2500e-04, 9.5000e-04, 9.7500e-04, 1.0000e-03])

In [8]:
[0.001 * i / 200 for i in range(1, 202, 5)]

[5e-06,
 3e-05,
 5.4999999999999995e-05,
 8e-05,
 0.000105,
 0.00013000000000000002,
 0.000155,
 0.00018,
 0.000205,
 0.00023,
 0.000255,
 0.00028000000000000003,
 0.000305,
 0.00033,
 0.00035500000000000006,
 0.00037999999999999997,
 0.00040500000000000003,
 0.00043000000000000004,
 0.000455,
 0.00048,
 0.000505,
 0.00053,
 0.000555,
 0.00058,
 0.000605,
 0.00063,
 0.000655,
 0.00068,
 0.0007050000000000001,
 0.00073,
 0.000755,
 0.00078,
 0.000805,
 0.00083,
 0.0008550000000000001,
 0.0008799999999999999,
 0.000905,
 0.0009299999999999999,
 0.000955,
 0.00098,
 0.001005]